## arcgis.env module
https://developers.arcgis.com/python/api-reference/arcgis.env.html

In [1]:
from arcgis.gis import GIS
from sgc import sgccat20lib
sgccat20lib.profile_storaged = "sgc20_sabanalarga"
gis = sgccat20lib.gisLogin()
print(gis.url)

https://arcgis.catasig.com.co/portal/


In [ ]:
from arcgis.env import *
print(active_gis.org_settings)

## arcgis.features module
https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html

In [ ]:
from arcgis.features import *
dictParams = sgccat20lib.ObtenerParametros()
parcel_service = dictParams["PARCEL_SERVICE_NAME"]
gis = sgccat20lib.gisLogin("profile")
ParcelService = gis.content.get(parcel_service)
urlParcelService = ParcelService.url
urlParcelService

In [ ]:
url_layer_table = "{0}/{1}".format(urlParcelService, sgccat20lib.getLayers()["TERRENO"]["ID"])
table = FeatureLayer(url_layer_table, gis=gis)
feature_set = table.query(
    where="1=1", out_fields="*", return_geometry=True,
    result_record_count=3, return_all_records=False, out_sr=True)

In [ ]:
if feature_set.features[0].get_value("retiredbyrecord"):
    print(feature_set.features[0].get_value("retiredbyrecord"))
else:
    print("No tiene valor el campo")

In [ ]:
if feature_set:
    feature = Feature(feature_set.features[0])
    print(type(feature))

In [ ]:
type(table)

In [ ]:
table.query(return_extent_only=True)

In [ ]:
flParcelService = FeatureLayerCollection.fromitem(ParcelService)
print(ParcelService.title, "es: ", type(flParcelService))
predio_table = next(iter(filter(lambda x: x.properties.name.upper()=="PREDIO".upper(), ParcelService.tables)), None)
table = next(iter(filter(lambda x: x.properties.name.upper()=="TERRENO".upper(), ParcelService.layers)), None)
print(table.properties.name, "es: ", type(table))
print(type(feature))
print("feature_set: ",type(feature_set))
print(feature_set.geometry_type)
feat_collect = FeatureCollection.from_featureset(feature_set)
print("FeatureCollection: ",type(feat_collect))

In [ ]:
feature_set.sdf

## arcgis.features.analysis:
https://developers.arcgis.com/python/api-reference/arcgis.features.analysis.html

In [ ]:
kwargs = { "return_geometry": True, "result_record_count": 3 }
datosPredio = sgccat20lib.get_features_layertable_where(
    gis,
    predio_table.url,
    "condicion_predio='NPH'",
    "*",
    None,
    ["LC_Terreno"],
    ["*"],
    **kwargs)

In [ ]:
rel_terreno = next(iter(filter(lambda x: x["name"].upper()=="LC_Terreno".upper(), datosPredio["Relationships"])), None)
if rel_terreno and len(rel_terreno["features"]) > 0:
    featuresTerreno = list(filter(lambda x: x["attributes"].get("retiredbyrecord") is None, rel_terreno["features"]))
    globalids = [f["attributes"]["globalid"] for f in featuresTerreno]
    GIDs_predio = tuple(globalids) if len(globalids) > 1 else "('{}')".format(globalids[0])
    where = "globalid in {0}".format(GIDs_predio)
    datosTerreno = table.query(where,return_geometry=True)

In [ ]:
import copy
fts_terreno = copy.deepcopy(datosTerreno)
feat_collect = FeatureCollection.from_featureset(fts_terreno)
print(type(feat_collect))
# create = analysis.aggregate_points()

In [ ]:
from arcgis.geometry import Geometry, Polygon

In [ ]:
polygon = Polygon(fts_terreno.features[0].geometry)
polygon